## Texts
For this particular model, I will use some ancient greek philosophy, from Marcus Aurelius, Epictetus, Seneca and Cicero

In [3]:
import requests

In [4]:
urls=[
    'https://www.gutenberg.org/cache/epub/2680/pg2680.txt', #meditations
    'https://www.gutenberg.org/cache/epub/10661/pg10661.txt', #epictetus
    'https://www.gutenberg.org/cache/epub/56075/pg56075.txt', #seneca1
    'https://www.gutenberg.org/cache/epub/3794/pg3794.txt', #seneca2
    'https://www.gutenberg.org/cache/epub/14988/pg14988.txt', #cicero1
    'https://www.gutenberg.org/cache/epub/58418/pg58418.txt', #cicero2
    'https://www.gutenberg.org/cache/epub/50692/pg50692.txt'  #cicero3
]

In [5]:
full_text = ''

In [6]:
import re
pattern = r"\*\*\* START OF THE PROJECT GUTENBERG EBOOK.*?\*\*\*\s*(.*)"

In [7]:
for url in urls:
    cur_text=requests.get(url).content.decode('utf-8')
    cur_text=re.search(pattern, cur_text, re.DOTALL) # avoid the beginning of each gutenberg ebook
    full_text+=cur_text.group(1)

In [8]:
len(full_text)

4458638

After combining text for all of the texts, it is now time for tokenization.
Tokenization is inspired by the video of the great Andrej Karpathy: https://www.youtube.com/watch?v=zduSFxRajkE&t=154s&ab_channel=AndrejKarpathy

In [23]:
vocab=[]
vocab_size = 300

In [24]:
for i in range(256): #first 256 unicodes characters for vocab
    vocab.append(i)

In [25]:
id_to_tokens={}
tokens_to_ids={}
id_to_tokens[0] = '<UNK>'
tokens_to_ids['<UNK>'] = 0
for i in range(1,256):
    id_to_tokens[i]=chr(i)
    tokens_to_ids[chr(i)]=i

In [26]:
tokenized_text = []
for char in full_text:
    char_code = ord(char)
    if char_code < 256:
        tokenized_text.append(char_code)
    else:
        tokenized_text.append(0)

In [27]:
from collections import Counter
import gc
def get_tokens(tokenized_text,vocab_size,id_to_tokens,tokens_to_ids):
    if vocab_size>len(id_to_tokens):
        pairs=Counter()
        for i in range(len(tokenized_text)-1):
            pairs[(tokenized_text[i],tokenized_text[i+1])]+=1

        if not pairs:
            return tokenized_text, id_to_tokens, tokens_to_ids
        most_common = pairs.most_common(1)[0][0]
        del pairs
        gc.collect() # need to force this garbage collector, as the memory might get crazy
        new_idx = len(id_to_tokens)
        new_token = id_to_tokens[most_common[0]]+id_to_tokens[most_common[1]]
        id_to_tokens[new_idx]=new_token
        tokens_to_ids[new_token]=new_idx
        i=0
        new_tokens=[]
        while i<len(tokenized_text):
            if i < len(tokenized_text) - 1 and  (tokenized_text[i],tokenized_text[i+1])==most_common:
                new_tokens.append(new_idx)
                i+=2
            else: 
                new_tokens.append(tokenized_text[i])
                i+=1
        return get_tokens(new_tokens,vocab_size,id_to_tokens,tokens_to_ids)
    
    else: return tokenized_text

In [28]:
new_tokenized_tex = get_tokens(tokenized_text,vocab_size,id_to_tokens,tokens_to_ids)

In [22]:
p=''
for i in new_tokenized_tex[:100]:
    p+=id_to_tokens[i]
print(p)

MEDITATIONS

By Marcus Aurelius




CONTENTS


     NOTES

     INTRODUCTION

     FIR


This solution works! But the recursion seems a bit messy.

In [16]:
from collections import Counter
import gc
def get_tokens_v2(tokenized_text,vocab_size,id_to_tokens,tokens_to_ids):
    for i in range(vocab_size-len(id_to_tokens)):
        pairs=Counter()
        for j in range(len(tokenized_text)-1):
            pairs[(tokenized_text[j],tokenized_text[j+1])]+=1

        if not pairs:
            return tokenized_text, id_to_tokens, tokens_to_ids
        most_common = pairs.most_common(1)[0][0]
        del pairs
        gc.collect() # need to force this garbage collector, as the memory might get crazy
        new_idx = len(id_to_tokens)
        new_token = id_to_tokens[most_common[0]]+id_to_tokens[most_common[1]]
        id_to_tokens[new_idx]=new_token
        tokens_to_ids[new_token]=new_idx
        j=0
        new_tokens=[]
        while j<len(tokenized_text):
            if j < len(tokenized_text) - 1 and  (tokenized_text[j],tokenized_text[j+1])==most_common:
                new_tokens.append(new_idx)
                j+=2
            else: 
                new_tokens.append(tokenized_text[j])
                j+=1
        tokenized_text = new_tokens
    return tokenized_text
        

In [17]:
new_tokenized_text = get_tokens_v2(tokenized_text,vocab_size,id_to_tokens,tokens_to_ids) 

In [19]:
p=''
for i in new_tokenized_text[:100]:
    p+=id_to_tokens[i]
print(p)

MEDITATIONS

By Marcus Aurelius




CONTENTS


     NOTES

     INTRODUCTION

     FIRST BOOK

     SECOND BOOK

     THIRD BOO


This solution is a bit clearer and slightly faster!